In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Parameters
image_size = 64
batch_size = 128
nz = 100  # Latent vector size
lr = 0.0002
beta1 = 0.5
epochs = 10

In [5]:
# Dataset path
dataset_path = "/Users/radistavrev/Documents/NBU/DCGAN/Data/Images"

In [6]:
# Image transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [7]:
# Load dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.model(x)

In [9]:
generator = Generator().to(device)

In [10]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [11]:
discriminator = Discriminator().to(device)

In [12]:
# Loss and Optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training
for epoch in range(epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)
        
        # Train Discriminator
        optimizer_d.zero_grad()
        outputs = discriminator(real_images).view(-1, 1)
        loss_real = criterion(outputs, real_labels)
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach()).view(-1, 1)
        loss_fake = criterion(outputs, fake_labels)
        loss_d = loss_real + loss_fake
        loss_d.backward()
        optimizer_d.step()
        
        # Train Generator
        optimizer_g.zero_grad()
        outputs = discriminator(fake_images).view(-1, 1)
        loss_g = criterion(outputs, real_labels)
        loss_g.backward()
        optimizer_g.step()
        
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}], Step [{i}/{len(dataloader)}], Loss_D: {loss_d.item()}, Loss_G: {loss_g.item()}")

Epoch [0/10], Step [0/161], Loss_D: 1.445210576057434, Loss_G: 3.4298646450042725
Epoch [0/10], Step [100/161], Loss_D: 0.30275627970695496, Loss_G: 5.944714546203613
Epoch [1/10], Step [0/161], Loss_D: 0.8487398624420166, Loss_G: 0.9762681126594543
Epoch [1/10], Step [100/161], Loss_D: 1.2034375667572021, Loss_G: 5.120983123779297
Epoch [2/10], Step [0/161], Loss_D: 0.4397053122520447, Loss_G: 3.165365219116211
Epoch [2/10], Step [100/161], Loss_D: 0.5155609250068665, Loss_G: 2.1475789546966553
Epoch [3/10], Step [0/161], Loss_D: 0.6021591424942017, Loss_G: 5.035395622253418
Epoch [3/10], Step [100/161], Loss_D: 1.209162950515747, Loss_G: 5.547321796417236
Epoch [4/10], Step [0/161], Loss_D: 0.8533746600151062, Loss_G: 5.266435623168945
Epoch [4/10], Step [100/161], Loss_D: 0.5946661233901978, Loss_G: 3.557684898376465
Epoch [5/10], Step [0/161], Loss_D: 0.6350954174995422, Loss_G: 2.2336785793304443


In [ ]:
# Generate Images
generator.eval()
noise = torch.randn(16, nz, 1, 1, device=device)
fake_images = generator(noise).cpu().detach()
fake_images = (fake_images + 1) / 2  # Normalize to [0,1]

In [ ]:
# Display generated images
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(np.transpose(fake_images[i].numpy(), (1, 2, 0)))
    ax.axis("off")

plt.show()

In [ ]:
num_generated_images = 100
output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Generate and Save 100 Images
generator.eval()
for i in range(num_generated_images):
    noise = torch.randn(1, nz, 1, 1, device=device)
    fake_image = generator(noise).cpu().detach()
    fake_image = (fake_image + 1) / 2  # Normalize to [0,1]
    vutils.save_image(fake_image, os.path.join(output_dir, f"generated_{i+1}.png"))

print(f"Generated {num_generated_images} images and saved to {output_dir}")